[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/https://github.com/cerr/CERR/blob/octave_dev/Jupyter_Notebooks/aws_cerr_load_and_view_data.ipynb)

## Create an environment with Octave, CERR and related dependencies

In [ ]:
%%bash
conda create -n cerrx
conda activate cerrx
conda install -y -c conda-forge octave oct2py octave_kernel=0.33.1 jupyterlab gxx cxx-compiler matplotlib 
conda install -y -c anaconda ipywidgets 
cd /home/studio-lab-user
git clone --single-branch --branch octave_dev https://www.github.com/cerr/CERR.git


## Install Octave packages (equivalent to Matlab toolboxes)

In [ ]:
%%octave
pkg install -forge image
pkg install -forge io
pkg install -forge statistics
pkg install https://gitlab.com/mtmiller/octave-pythonic/-/archive/v0.0.1/octave-pythonic-v0.0.1.tar.gz
pkg list

In [ ]:
%load_ext oct2py.ipython

## Add CERR to Octave path

In [61]:
%%octave -t /home/studio-lab-user/oct2py_io
pkg load image
pkg load io
pkg load statistics
pkg load pythonic
cerrPath = '/home/studio-lab-user/CERR';
cd(cerrPath)
addToPath2(cerrPath)

Adding to path ... /home/studio-lab-user/CERR/CERR_Data_Extraction

Adding to path ... /home/studio-lab-user/CERR/CERR_core

Adding to path ... /home/studio-lab-user/CERR/IMRTP

Adding to path ... /home/studio-lab-user/CERR/Jupyter_Notebooks

Adding to path ... /home/studio-lab-user/CERR/ML_Dicom

Adding to path ... /home/studio-lab-user/CERR/Python_packages

Adding to path ... /home/studio-lab-user/CERR/Unit_Testing

## Load planC from file

In [62]:
%%octave -t /home/studio-lab-user/oct2py_io
cerrFileName = fullfile(cerrPath,'Unit_Testing','data_for_cerr_tests', 'CERR_plans', 'lung_ex1_20may03.mat.bz2')
planC = loadPlanC(cerrFileName,tempdir);


cerrFileName = /home/studio-lab-user/CERR/Unit_Testing/data_for_cerr_tests/CERR_plans/lung_ex1_20may03.mat.bz2

CERR>>  Decompressing lung_ex1_20may03.mat.bz2...

ans = 0

CERR>>  Loading lung_ex1_20may03.mat.bz2...

CERR>>  Loaded lung_ex1_20may03.mat.bz2...

## Access scan volume and structures from planC

In [63]:
%%octave

strNameC = {'HEART', 'LIVER', 'LUNG_CONTRA',...
              'LUNG_IPSI', 'SPINAL_CORD', 'PTV1'};

%Get scan array
indexS = planC{end};
scanNum = 1;
ctOffset = planC{indexS.scan}(scanNum).scanInfo(1).CTOffset;
scanArray = single(getScanArray(scanNum,planC)) - ctOffset;

%Get structure labels & masks
numStructs = length(planC{indexS.structures});
structNameC = {planC{indexS.structures}.structureName};
for strNum = 1:length(strNameC)
    strx = strNameC{strNum};
    idx = getMatchingIndex(strx,structNameC,'EXACT');
    mask3M = getStrMask(idx, planC);
    maskC{strNum} = mask3M;
end

In [64]:
%octave_pull maskC strNameC scanArray

## Visualize scan and structures

In [65]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import ipywidgets as widgets

dx, dy = 1, 1

x = np.arange(0, 255, dx)
y = np.arange(0, 255, dy)
extent = np.min(x), np.max(x), np.min(y), np.max(y)

clear_output(wait=True)

def window_image(image, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    window_image = image.copy()
    window_image[window_image < img_min] = img_min
    window_image[window_image > img_max] = img_max
    
    return window_image

def show_axial_slice(slcNum):
    clear_output(wait=True)
    print('Slice '+str(slcNum))
    if 'fig' in locals():
        fig.remove()
    fig, ax = plt.subplots()
    window_center = -500
    window_width = 1500
    windowed_img = window_image(scanArray[:,:,slcNum-1],
                                window_center,window_width)
    im1 = ax.imshow(windowed_img, cmap=plt.cm.gray, alpha=1,
                    interpolation='nearest', extent=extent)
    
    cmaps = [plt.cm.YlOrBr.copy(),plt.cm.Oranges.copy(),plt.cm.YlGn.copy(),plt.cm.Blues.copy(), \
             plt.cm.Purples.copy(),plt.cm.Greens.copy()]

    for maskNum in range(0,6,1):
        mask_cmap = cmaps[maskNum]
        mask_cmap.set_under('k', alpha=0)
        im2 = ax.imshow(maskC[0,maskNum][:,:,slcNum-1], 
                        cmap=mask_cmap, alpha=.8, extent=extent,
                        interpolation='none', clim=[0.5, 1])        
    plt.show()

slice_slider = widgets.IntSlider(value=30,min=1,max=79,step=1)
outputSlc = widgets.Output()

display(slice_slider, outputSlc)

def update_slice(change):
    with outputSlc:
        show_axial_slice(change['new'])

slice_slider.observe(update_slice, names='value')

IntSlider(value=30, max=79, min=1)

Output()